In [1]:
import warnings
warnings.filterwarnings('ignore')

import glob
import iris
from iris.experimental.equalise_cubes import equalise_attributes
import iris.coord_categorisation
import xarray as xr

In [2]:
infiles = glob.glob('/g/data/ua6/DRSv3/CMIP5/CCSM4/historical/mon/ocean/r1i1p1/thetao/latest/thetao_Omon_CCSM4_historical_r1i1p1_??????-??????.nc')
infiles.sort()

## xarray

In [3]:
df = xr.open_mfdataset(infiles)

In [4]:
df.thetao

<xarray.DataArray 'thetao' (time: 6012, lev: 60, j: 384, i: 320)>
dask.array<shape=(6012, 60, 384, 320), dtype=float32, chunksize=(120, 60, 384, 320)>
Coordinates:
  * lev      (lev) float64 5.0 15.0 25.0 35.0 ... 4.875e+03 5.125e+03 5.375e+03
  * j        (j) int32 1 2 3 4 5 6 7 8 9 ... 376 377 378 379 380 381 382 383 384
  * i        (i) int32 1 2 3 4 5 6 7 8 9 ... 312 313 314 315 316 317 318 319 320
    lat      (j, i) float32 -79.22052 -79.22052 -79.22052 ... 72.18933 72.185974
    lon      (j, i) float32 320.5625 321.6875 322.8125 ... 319.35068 319.7835
  * time     (time) object 0800-01-16 00:00:00 ... 1300-12-16 12:00:00
Attributes:
    standard_name:     sea_water_potential_temperature
    long_name:         Sea Water Potential Temperature
    units:             K
    original_name:     TEMP
    comment:           TEMP no change, units from C to K
    original_units:    degC
    history:           2011-11-29T21:51:13Z altered by CMOR: Converted units ...
    cell_methods:      

In [4]:
thetao_annual = df.thetao.groupby('time.year').mean(dim='time')

In [5]:
thetao_annual

<xarray.DataArray 'thetao' (year: 156, lev: 60, j: 384, i: 320)>
dask.array<shape=(156, 60, 384, 320), dtype=float32, chunksize=(1, 60, 384, 320)>
Coordinates:
  * lev      (lev) float64 5.0 15.0 25.0 35.0 ... 4.875e+03 5.125e+03 5.375e+03
  * j        (j) int32 1 2 3 4 5 6 7 8 9 ... 376 377 378 379 380 381 382 383 384
  * i        (i) int32 1 2 3 4 5 6 7 8 9 ... 312 313 314 315 316 317 318 319 320
    lat      (j, i) float32 -79.22052 -79.22052 -79.22052 ... 72.18933 72.185974
    lon      (j, i) float32 320.5625 321.6875 322.8125 ... 319.35068 319.7835
  * year     (year) int64 1850 1851 1852 1853 1854 ... 2001 2002 2003 2004 2005

In [6]:
thetao_annual.values

array([[[[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [271.34683, 271.34644, 271.34677, ...,       nan,       nan,
                nan],
         ...,
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [271.34735, 271.34732, 271.34738, ...,       nan,       nan,
                nan],
         ...,
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,   

## Iris

In [3]:
cube_list = iris.load(infiles)

In [4]:
cube_list

[<iris 'Cube' of sea_water_potential_temperature / (K) (time: 120; depth: 60; cell index along second dimension: 384; cell index along first dimension: 320)>,
<iris 'Cube' of sea_water_potential_temperature / (K) (time: 120; depth: 60; cell index along second dimension: 384; cell index along first dimension: 320)>,
<iris 'Cube' of sea_water_potential_temperature / (K) (time: 120; depth: 60; cell index along second dimension: 384; cell index along first dimension: 320)>,
<iris 'Cube' of sea_water_potential_temperature / (K) (time: 120; depth: 60; cell index along second dimension: 384; cell index along first dimension: 320)>,
<iris 'Cube' of sea_water_potential_temperature / (K) (time: 120; depth: 60; cell index along second dimension: 384; cell index along first dimension: 320)>,
<iris 'Cube' of sea_water_potential_temperature / (K) (time: 120; depth: 60; cell index along second dimension: 384; cell index along first dimension: 320)>,
<iris 'Cube' of sea_water_potential_temperature / (

In [5]:
equalise_attributes(cube_list)
#iris.util.unify_time_units(cube)
cube = cube_list.concatenate_cube()

In [6]:
cube

<iris 'Cube' of sea_water_potential_temperature / (K) (time: 1872; depth: 60; cell index along second dimension: 384; cell index along first dimension: 320)>

In [7]:
iris.coord_categorisation.add_year(cube, 'time')
iris.coord_categorisation.add_month(cube, 'time')

In [ ]:
cube = cube.aggregated_by(['year'], iris.analysis.MEAN)